In [1]:
# IR HW5 Simple Mixture Model MAP@5000
path = './ntust-ir-2020_hw5_new/'
import numpy as np
import time
import random
from tqdm import tqdm
import numba as nb
from collections import Counter
from math import log

In [2]:
# data of doc and query
doc = []
query = []
# length of each doc
doc_len = []
# Name of doc and query
doc_list = []
query_list = []
# TF of doc and query
tf_doc = []
tf_q = []
# BM25 query vocabulary
bm25_vocabulary = set()
# Document IDF
doc_idf = dict()
# -----------------------------------------------------------------------------
# get doc files and query files
with open(path + 'query_list.txt', 'r') as fq:
    line = fq.read().splitlines()
    for l in line:
        query_list.append(l)
        f_temp = open(path + 'queries/{}.txt'.format(l))
        temp_query = f_temp.read().split()
        tf_q.append(Counter(temp_query))
        for w in temp_query:
            bm25_vocabulary.add(w)
        query.append(temp_query)
        f_temp.close()
with open(path + 'doc_list.txt', 'r') as fd:
    line = fd.read().splitlines()
    for l in tqdm(line):
        doc_list.append(l)
        f_temp = open(path + 'docs/{}.txt'.format(l))
        temp_doc = f_temp.read().split()
        for k in Counter(temp_doc).keys():
            if(k in doc_idf):
                doc_idf[k] += 1
            else: doc_idf[k] = 1
        tf_doc.append(Counter(temp_doc))
        doc.append(temp_doc)
        doc_len.append(len(temp_doc))
        f_temp.close()

100%|██████████| 30000/30000 [02:38<00:00, 189.44it/s]


In [3]:
# BackGroud variable
bg_len = np.sum(doc_len)
# All doc, query and vocabulary length
d_len = len(doc)
q_len = len(query)
# bm vocabulary to list
bm25_vocabulary = list(bm25_vocabulary)
bm25_len = len(bm25_vocabulary)

In [4]:
# build vocabulary
vocabulary = set()
for w in tqdm(bm25_vocabulary):
    if(not(w in doc_idf)):
        doc_idf[w] = 0
        for d in range(d_len):
            if(w in tf_doc[d]):
                doc_idf[w] += 1
for w in doc_idf.keys():
    if(doc_idf[w] > 5):
        vocabulary.add(w)
vocabulary = list(vocabulary)
w_len = len(vocabulary)
print(w_len)

100%|██████████| 324/324 [00:00<00:00, 328487.91it/s]31328



In [5]:
voc_counter = Counter(vocabulary)

In [6]:
# Query ULM
q_ulm = []
for q in range(q_len):
    temp = dict()
    ql = len(query[q])
    for k,v in tf_q[q].items():
        if(voc_counter[k] > 0):
            temp[k] = v / ql
    q_ulm.append(temp)
# Doc ULM
d_ulm = []
for d in range(d_len):
    temp = dict()
    for k,v in tf_doc[d].items():
        if(voc_counter[k] > 0):
            temp[k] = v / doc_len[d]
    d_ulm.append(temp)

In [7]:
# Vocabulary index
voc_index = dict()
for w in range(w_len):
    voc_index[vocabulary[w]] = w

In [8]:
# Probability BackGround Model
P_bg = np.zeros(w_len)
for w in tqdm(range(w_len)):
    s = 0
    for d in range(d_len):
        s += tf_doc[d][vocabulary[w]]
    P_bg[w] = s / bg_len

100%|██████████| 31328/31328 [08:09<00:00, 63.97it/s]


In [9]:
# IDF
def IDF(idf, tf_key, bm25_voc, dl, bm25l):
    for d in tqdm(range(dl)):
        for v in range(bm25l):
            if(tf_key[d][bm25_voc[v]] > 0):
                idf[v] += 1
    return idf

In [10]:
# BM25 model variable
K1 = 0.8
b = 0.7
N = d_len
avg_dl = np.average(doc_len)

In [11]:
# BM25 IDF
idf_doc = np.zeros(len(bm25_vocabulary))
idf_doc = IDF(idf_doc, tf_doc, bm25_vocabulary, d_len, bm25_len)

100%|██████████| 30000/30000 [00:02<00:00, 14970.15it/s]


In [12]:
# B25 Model
def BM25(sim, k1, b_, N_, avg, dl_list, dl, ql, tf_key, Query, idf_, bm_voc):
    for q in tqdm(range(ql)):
        for d in range(dl):
            sum = 0.0
            for w in Query[q]:
                w_tf = tf_key[d][w]
                ni_loc = bm_voc.index(w)
                ni = idf_[ni_loc]
                sum_tmp = (k1 + 1) * w_tf / (k1 * ((1 - b) + b * dl_list[d] / avg) + w_tf)
                sum_tmp *= log((N_ - ni + 0.5) / (ni + 0.5))
                sum += sum_tmp
            sim[q][d] = sum
    return sim

In [13]:
# BM25 Sim
bm_sim = np.zeros([q_len, d_len], dtype=float)
bm_sim = BM25(bm_sim, K1, b, N, avg_dl, doc_len, d_len, q_len, tf_doc, query, idf_doc, bm25_vocabulary)

100%|██████████| 150/150 [00:46<00:00,  3.19it/s]


In [93]:
# SMM variable
SMM = int(5)
Rq = np.zeros([q_len, SMM], dtype=np.int64)
for q in range(q_len):
    arg = np.argsort(-bm_sim[q])[:SMM]
    for s in range(SMM):
        Rq[q][s] = arg[s]

In [94]:
# Simple Mixture Model initialization
PSMM = np.zeros([q_len, w_len])
PTsmm = np.zeros([q_len, w_len])
# Random
for q in tqdm(range(q_len)):
    r_sum = 0
    for w in range(w_len):
        p_temp = random.uniform(1.0, 10.0)
        PSMM[q][w] = p_temp
        r_sum += p_temp
    for w in range(w_len):
        PSMM[q][w] /= r_sum

100%|██████████| 150/150 [00:05<00:00, 26.79it/s]


In [95]:
# SMM E_step
def E_step(tsmm, alpha_, psmm, p_bg):
    mol = (1 - alpha_) * psmm
    den = alpha_ * p_bg
    den = np.sum(np.vstack((mol, den)), axis=0)
    tsmm = np.divide(mol, den)
    return tsmm

In [96]:
# Build Relevance c(w, d)
def RC(r_w, rele_doc, rel_len, cq):
    for d in range(rel_len):
        for w in range(w_len):
            r_w[d][w] = tf_doc[rele_doc[cq][d]][vocabulary[w]]
    return r_w

In [97]:
# SMM M_step
def M_step(psmm, tsmm, r_w):
    den = np.empty(w_len)
    den.fill(np.sum(r_w))
    mol = np.sum(r_w, axis=0)
    mol = np.multiply(mol, tsmm)
    psmm = np.divide(mol, den)
    return psmm

In [98]:
iter = 30
alpha = 0.4
for i in tqdm(range(iter)):
    for q in range(q_len):
        PTsmm[q] = E_step(PTsmm[q], alpha, PSMM[q], P_bg)
        RC_array = np.zeros([SMM, w_len])
        RC_array = RC(RC_array, Rq, SMM, q)
        PSMM[q] = M_step(PSMM[q], PTsmm[q], RC_array)

100%|██████████| 30/30 [09:34<00:00, 19.15s/it]


In [99]:
# KL variable
kl_alpha = 0.3
kl_beta = 0.5
kl_lamda = 0.5

In [100]:
def KL_D(KL, alpha_, beta_, psmm, lamda):
    for q in tqdm(range(q_len)):
        ql = len(query[q])
        p_w_q = np.zeros(w_len)
        for k in tf_q[q].keys():
            if(voc_counter[k] > 0):
                p_w_q[voc_index[k]] = q_ulm[q][k] * alpha_
        p_w_q = np.vstack((p_w_q, (beta_ * psmm[q])))
        p_w_q = np.vstack((p_w_q, ((1 - alpha_ - beta_) * P_bg)))
        p_w_q = np.sum(p_w_q, axis=0)
        for d in range(d_len):
            dl = doc_len[d]
            p_w_d = np.zeros(w_len)
            for k in tf_doc[d].keys():
                if(voc_counter[k] > 0):
                    p_w_d[voc_index[k]] = lamda * d_ulm[d][k]
            p_w_d = np.log(np.sum(np.vstack((p_w_d, ((1 - lamda) * P_bg))), axis=0))
            KL[q][d] = -np.sum(np.multiply(p_w_q, p_w_d))
    return KL

In [101]:
kl_div = np.zeros([q_len, d_len])
kl_div = KL_D(kl_div, kl_alpha, kl_beta, PSMM, kl_lamda)

100%|██████████| 150/150 [28:07<00:00, 11.25s/it]


In [102]:
np.save('KL.npy', kl_div)

In [103]:
# Create output file
fp = open("SMM.txt", "w")
fp.write("Query,RetrievedDocuments\n")
for i in range(q_len):
    fp.write('{},'.format(query_list[i]))
    for s in np.argsort(kl_div[i])[:5000]:
        fp.write(doc_list[s] + ' ')
    fp.write('\n')
fp.close()